In [1]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
embeddings = AzureOpenAIEmbeddings(
openai_api_type="azure",
openai_api_version=os.environ["OPENAI_API_EMBEDDING_VERSION"],
openai_api_key=os.environ["OPENAI_API_EMBEDDING_KEY"],
azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDING_ENDPOINT"],
deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
model=os.environ["AZURE_OPENAI_EMBEDDING_MODEL"],
validate_base_url=True,
)


In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec
import time

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
vector_store = PineconeVectorStore(index=pc.Index("langstuffindex"), embedding=embeddings)

In [ ]:
# SKIP this, only for local DB loading deprecated. 
from langchain_neo4j import Neo4jVector

url = "bolt://localhost:7687"
username = "neo4j"
password = "password"


vector_store = Neo4jVector.from_existing_index(
    embeddings,
    url=url,
    username=username,
    password=password,
    index_name="neo4j"
)

2025-04-05 15:18:19,663 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


In [6]:
import logging
# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__)

In [7]:
!pip install html2text
import html2text
import aiohttp
import asyncio
from bs4 import BeautifulSoup
from langchain_core.documents import Document
import json

async def parse_url_to_doc(url: str) -> str:
    try:
        timeout = aiohttp.ClientTimeout(total=10)  # Set timeout to 3 seconds
        async with aiohttp.ClientSession() as session:
            async with session.get(url, timeout= timeout) as response:
                if response.status != 200:
                    logger.warning(f"Failed to fetch {url}: Status code {response.status}")
                    return None
                html_content = await response.text()
                soup = BeautifulSoup(html_content, 'html.parser')
                target_div = soup.find('div', class_= "theme-doc-markdown markdown") #langchain

                if not target_div:
                    target_div = soup.find('article') #langraph
                
                if not target_div:
                    target_div = soup.find("div", id="content-area")
                
                if not target_div:
                    target_div = soup.find("textarea", id="read-only-cursor-text-area")

                if not target_div:
                    return None

                return Document(page_content=html2text.html2text(str(target_div)), metadata={"source": url})
    except Exception as e:
        logger.error(f"Error in parsing {url}: {e}")
    

In [ ]:
doc = await parse_url_to_doc("https://modelcontextprotocol.io/docs/concepts/tools")

print(doc)

page_content='Concepts

# Tools

Enable LLMs to perform actions through your server

Tools are a powerful primitive in the Model Context Protocol (MCP) that enable
servers to expose executable functionality to clients. Through tools, LLMs can
interact with external systems, perform computations, and take actions in the
real world.

Tools are designed to be **model-controlled** , meaning that tools are exposed
from servers to clients with the intention of the AI model being able to
automatically invoke them (with a human in the loop to grant approval).

##

​

Overview

Tools in MCP allow servers to expose executable functions that can be invoked
by clients and used by LLMs to perform actions. Key aspects of tools include:

  * **Discovery** : Clients can list available tools through the `tools/list` endpoint
  * **Invocation** : Tools are called using the `tools/call` endpoint, where servers perform the requested operation and return results
  * **Flexibility** : Tools can range from s

C:\Users\kanis\AppData\Local\Temp\ipykernel_1420\3532148005.py:1: RuntimeWarning: coroutine 'parse_url_to_doc' was never awaited
  doc = await parse_url_to_doc("https://modelcontextprotocol.io/docs/concepts/tools")


In [31]:
with open('../../webscraping/visited_urls_langchain.json', 'r') as f:
    urls = json.load(f)

    # Generator function to yield 10 items at a time
    def batch_iterator(iterable, batch_size):
        for i in range(0, len(iterable), batch_size):
            yield iterable[i:i + batch_size]

    batch_size = 10

    # Create the iterator
    iterator = batch_iterator(urls, batch_size)

    documents = []

    # Use the iterator
    for batch in iterator:
        tasks = [parse_url_to_doc(url) for url in batch]
        results = await asyncio.gather(*tasks)
        documents.append(results)
    documents = [item for sublist in documents for item in sublist]
    documents =  [item for item in documents if item != None]

2025-04-06 14:42:18,987 - WARNING - Failed to fetch https://python.langchain.com/docs/expression_language/streaming: Status code 404
2025-04-06 14:42:19,214 - WARNING - Failed to fetch https://python.langchain.com/docs/expression_language/how_to/inspect: Status code 404
2025-04-06 14:42:19,718 - WARNING - Failed to fetch https://python.langchain.com/docs/expression_language/how_to/decorator: Status code 404


In [32]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name="cl100k_base",
    chunk_size=1500, chunk_overlap=100
)
doc_splits = text_splitter.split_documents(documents)

len(doc_splits)

1800

In [33]:
doc_len=0
import tiktoken
for doc in doc_splits:
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(doc.page_content))
    doc_len += num_tokens
    
print(doc_len)

1911028


In [34]:
import time
iterator = batch_iterator(doc_splits, 220)

for doc in iterator:
    try:
        vector_store.add_documents(doc)
    except Exception as ex:
        logger.info(f"{ex} caught, adding a 62 second sleep and retrying again")
        time.sleep(62)
        vector_store.add_documents(doc)

2025-04-06 14:44:28,979 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-06 14:44:54,298 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-06 14:45:16,562 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-06 14:45:47,629 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-06 14:46:18,875 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-06 14:46:53,795 - INFO - HT

In [12]:
vector_store.similarity_search_with_relevance_scores("implement a langgraph swarm agent", k=10)

2025-04-06 13:58:57,310 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


[(Document(id='0dc04ca9-5f69-48f7-81eb-d1ed937a78a9', metadata={'source': 'https://github.com/langchain-ai/langgraph-swarm-py/blob/main/README.md'}, page_content='# 🤖 LangGraph Multi-Agent Swarm\n\nA Python library for creating swarm-style multi-agent systems using\n[LangGraph](https://github.com/langchain-ai/langgraph). A swarm is a type of\n[multi-agent](https://langchain-ai.github.io/langgraph/concepts/multi_agent)\narchitecture where agents dynamically hand off control to one another based on\ntheir specializations. The system remembers which agent was last active,\nensuring that on subsequent interactions, the conversation resumes with that\nagent.\n\n[![Swarm](/langchain-ai/langgraph-swarm-\npy/raw/main/static/img/swarm.png)](/langchain-ai/langgraph-swarm-\npy/blob/main/static/img/swarm.png)\n\n## Features\n\n  * 🤖 **Multi-agent collaboration** \\- Enable specialized agents to work together and hand off context to each other\n  * 🛠️ **Customizable handoff tools** \\- Built-in too